In [23]:
import cv2
import numpy as np
import csv
from datetime import datetime

# === ARUCO DETECTOR SETUP (OpenCV ≥4.7) ===
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
aruco_params = cv2.aruco.DetectorParameters()
aruco_detector = cv2.aruco.ArucoDetector(aruco_dict, aruco_params)

def detect_aruco_scale(gray, aruco_detector, marker_id, marker_size_cm):
    corners, ids, _ = aruco_detector.detectMarkers(gray)
    if ids is not None and marker_id in ids:
        idx = np.where(ids == marker_id)[0][0]
        c = corners[idx][0]
        side_px = np.mean([
            np.linalg.norm(c[0]-c[1]),
            np.linalg.norm(c[1]-c[2]),
            np.linalg.norm(c[2]-c[3]),
            np.linalg.norm(c[3]-c[0])
        ])
        cm_per_pixel = marker_size_cm / side_px
        measured_marker_cm = side_px * cm_per_pixel
        return cm_per_pixel, measured_marker_cm, c
    return None, None, None


# === CONFIGURATION ===
use_webcam = True
video_path = "assets/video2.mp4"
cap = cv2.VideoCapture(0 if use_webcam else video_path)
if not cap.isOpened():
    raise IOError("Cannot open video / webcam.")

# Reference for fallback scaling
reference_width_cm  = 8.0
reference_height_cm = 5.0

marker_id_local = 0
marker_size_cm_local = 5.1
soft_mode = True
last_cm_per_pixel_local = None
marker_status_text = "No marker detected"

# === CSV LOGGING SETUP ===
timestamp_now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
csv_filename = f"bun_measurements_{timestamp_now}.csv"

with open(csv_filename, mode="w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Timestamp", "Frame Index", "Marker Status", "cm/pixel",
                     "Width (cm)", "Height (cm)", "Volume (cm^3)", "Margin of Error"])

frame_idx = 0

# === MAIN LOOP ===
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_idx += 1

    vis = frame.copy()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # --- detect ArUco marker ---
    cm_per_pixel, measured_marker_cm, marker_corners = detect_aruco_scale(
        gray, aruco_detector, marker_id_local, marker_size_cm_local
    )

    if marker_corners is not None:
        int_box = np.intp(marker_corners)
        cv2.polylines(vis, [int_box], True, (255, 0, 0), 2)
        cv2.putText(vis, f"Scale: {cm_per_pixel:.6f} cm/px", (10, 25),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200, 200, 0), 2)
        last_cm_per_pixel_local = cm_per_pixel
        marker_status_text = f"Marker OK (ID {marker_id_local})"
    else:
        if last_cm_per_pixel_local is not None and soft_mode:
            cm_per_pixel = last_cm_per_pixel_local
            marker_status_text = f"Marker lost - using last scale = {last_cm_per_pixel_local:.6f} cm/px"
        else:
            marker_status_text = "No marker - using reference tin size"
            h_img, w_img = gray.shape
            cm_per_pixel_x = reference_width_cm / w_img
            cm_per_pixel_y = reference_height_cm / h_img
            cm_per_pixel = (cm_per_pixel_x + cm_per_pixel_y) / 2

    # === OBJECT DETECTION ===
    _, obj_mask = cv2.threshold(gray, 60, 255, cv2.THRESH_BINARY)
    edited = frame.astype(np.float32)
    edited += 100
    edited = edited * 0.5 + 64
    edited = np.clip(edited, 0, 255).astype(np.uint8)
    edited_obj = frame.copy()
    edited_obj[obj_mask == 255] = edited[obj_mask == 255]

    gray2 = cv2.cvtColor(edited_obj, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray2, (9, 9), 0)
    sobel_x = cv2.Sobel(blur, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(blur, cv2.CV_64F, 0, 1, ksize=3)
    sobel = cv2.magnitude(sobel_x, sobel_y)
    sobel = cv2.convertScaleAbs(sobel)
    _, thresh = cv2.threshold(sobel, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    kernel = np.ones((7, 7), np.uint8)
    thresh_closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    contours, _ = cv2.findContours(thresh_closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # === MEASUREMENT ===
    if contours:
        largest = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(largest)
        bun_mask = np.zeros_like(gray2)
        cv2.drawContours(bun_mask, [largest], -1, 255, -1)

        diameters_px = []
        for col in range(x, x + w):
            col_pix = np.where(bun_mask[:, col] > 0)[0]
            if len(col_pix) > 1:
                diameters_px.append(col_pix.max() - col_pix.min())

        if len(diameters_px) > 10:
            diameters_px = np.array(diameters_px)
            avg_d = np.mean(diameters_px)
            std_d = np.std(diameters_px)
            diameters_cm = diameters_px * cm_per_pixel
            delta_x_cm = cm_per_pixel
            volume_cm3 = np.sum(np.pi * (diameters_cm / 2) ** 2 * delta_x_cm)

            rel_err_shape = std_d / avg_d if avg_d > 0 else 0
            rel_err_calib = 0.01
            err_total = np.sqrt(rel_err_shape**2 + rel_err_calib**2) * 100

            w_cm = w * cm_per_pixel
            h_cm = h * cm_per_pixel

            cv2.rectangle(vis, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(vis, f"Width: {w_cm:.2f} cm  Height: {h_cm:.2f} cm", (x, y - 25),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            cv2.putText(vis, f"Vol: {volume_cm3:.1f} cm³ (+/- {err_total:.1f}%)",
                        (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

            # === WRITE TO CSV ===
            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            with open(csv_filename, mode="a", newline="") as f:
                writer = csv.writer(f)
                writer.writerow([timestamp, frame_idx, marker_status_text,
                                 round(cm_per_pixel, 6), round(w_cm, 3), round(h_cm, 3),
                                 round(volume_cm3, 3), round(err_total, 3)])

    cv2.putText(vis, marker_status_text, (10, 70),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 200, 255), 2)

    cv2.imshow("Bun Volume Measurement (ArUco + CSV Logging)", vis)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

print(f"\n✅ Data saved to: {csv_filename}")


✅ Data saved to: bun_measurements_2025-11-07_18-49-31.csv


In [26]:
import cv2
import numpy as np
import csv
from datetime import datetime

# === ARUCO DETECTOR SETUP (OpenCV ≥4.7) ===
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
aruco_params = cv2.aruco.DetectorParameters()
aruco_detector = cv2.aruco.ArucoDetector(aruco_dict, aruco_params)

def detect_aruco_scale(gray, aruco_detector, marker_id, marker_size_cm):
    corners, ids, _ = aruco_detector.detectMarkers(gray)
    if ids is not None and marker_id in ids:
        idx = np.where(ids == marker_id)[0][0]
        c = corners[idx][0]
        side_px = np.mean([
            np.linalg.norm(c[0]-c[1]),
            np.linalg.norm(c[1]-c[2]),
            np.linalg.norm(c[2]-c[3]),
            np.linalg.norm(c[3]-c[0])
        ])
        cm_per_pixel = marker_size_cm / side_px
        measured_marker_cm = side_px * cm_per_pixel
        return cm_per_pixel, measured_marker_cm, c
    return None, None, None


# === CONFIGURATION ===
use_webcam = True
video_path = "assets/video2.mp4"
cap = cv2.VideoCapture(0 if use_webcam else video_path)
if not cap.isOpened():
    raise IOError("Cannot open video / webcam.")

# Reference for fallback scaling
reference_width_cm  = 8.0
reference_height_cm = 5.0

marker_id_local = 0
marker_size_cm_local = 5.1
soft_mode = True
last_cm_per_pixel_local = None
marker_status_text = "No marker detected"

# === CSV LOGGING SETUP ===
timestamp_now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
csv_filename = f"object_measurements_{timestamp_now}.csv"

with open(csv_filename, mode="w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Timestamp", "Frame Index", "Marker Status", "cm/pixel",
                     "Width (cm)", "Height (cm)", "Volume (cm^3)", "Margin of Error (%)"])

frame_idx = 0

# === MAIN LOOP ===
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_idx += 1

    vis = frame.copy()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # --- detect ArUco marker ---
    cm_per_pixel, measured_marker_cm, marker_corners = detect_aruco_scale(
        gray, aruco_detector, marker_id_local, marker_size_cm_local
    )

    if marker_corners is not None:
        int_box = np.intp(marker_corners)
        cv2.polylines(vis, [int_box], True, (255, 0, 0), 2)
        cv2.putText(vis, f"Scale: {cm_per_pixel:.6f} cm/px", (10, 25),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200, 200, 0), 2)
        last_cm_per_pixel_local = cm_per_pixel
        marker_status_text = f"Marker OK (ID {marker_id_local})"
    else:
        if last_cm_per_pixel_local is not None and soft_mode:
            cm_per_pixel = last_cm_per_pixel_local
            marker_status_text = f"Marker lost - using last scale = {last_cm_per_pixel_local:.6f} cm/px"
        else:
            marker_status_text = "No marker - using reference tin size"
            h_img, w_img = gray.shape
            cm_per_pixel_x = reference_width_cm / w_img
            cm_per_pixel_y = reference_height_cm / h_img
            cm_per_pixel = (cm_per_pixel_x + cm_per_pixel_y) / 2

    # === OBJECT DETECTION ===
    _, obj_mask = cv2.threshold(gray, 60, 255, cv2.THRESH_BINARY)
    edited = frame.astype(np.float32)
    edited += 100
    edited = edited * 0.5 + 64
    edited = np.clip(edited, 0, 255).astype(np.uint8)
    edited_obj = frame.copy()
    edited_obj[obj_mask == 255] = edited[obj_mask == 255]

    gray2 = cv2.cvtColor(edited_obj, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray2, (9, 9), 0)
    sobel_x = cv2.Sobel(blur, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(blur, cv2.CV_64F, 0, 1, ksize=3)
    sobel = cv2.magnitude(sobel_x, sobel_y)
    sobel = cv2.convertScaleAbs(sobel)
    _, thresh = cv2.threshold(sobel, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    kernel = np.ones((7, 7), np.uint8)
    thresh_closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    contours, _ = cv2.findContours(thresh_closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # === MEASUREMENT ===
    if contours:
        largest = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(largest)
        bun_mask = np.zeros_like(gray2)
        cv2.drawContours(bun_mask, [largest], -1, 255, -1)

        diameters_px = []
        for col in range(x, x + w):
            col_pix = np.where(bun_mask[:, col] > 0)[0]
            if len(col_pix) > 1:
                diameters_px.append(col_pix.max() - col_pix.min())

        if len(diameters_px) > 10:
            diameters_px = np.array(diameters_px)
            avg_d = np.mean(diameters_px)
            std_d = np.std(diameters_px)
            diameters_cm = diameters_px * cm_per_pixel
            delta_x_cm = cm_per_pixel
            volume_cm3 = np.sum(np.pi * (diameters_cm / 2) ** 2 * delta_x_cm)

            # --- IMPROVED ERROR CALCULATION ---
            std_d_cm = np.std(diameters_px) * cm_per_pixel
            avg_d_cm = np.mean(diameters_px) * cm_per_pixel
            rel_err_shape = (std_d_cm / avg_d_cm) if avg_d_cm > 0 else 0
            rel_err_calib = 0.005  # 0.5% calibration uncertainty
            err_total = np.sqrt(rel_err_shape**2 + rel_err_calib**2) * 100

            # Optional: known real dimensions for comparison
            # (you can comment these out or update for your test object)
            true_width_cm = 17.5
            true_height_cm = 6.5
            w_cm = w * cm_per_pixel
            h_cm = h * cm_per_pixel
            width_diff_pct = abs((w_cm - true_width_cm) / true_width_cm) * 100
            height_diff_pct = abs((h_cm - true_height_cm) / true_height_cm) * 100
            physical_err_pct = (width_diff_pct + height_diff_pct) / 2

            # Combine both sources of uncertainty
            # err_total = np.sqrt(err_total**2 + physical_err_pct**2)

            # --- DISPLAY ---
            cv2.rectangle(vis, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(vis, f"Width: {w_cm:.2f} cm  Height: {h_cm:.2f} cm", (x, y - 25),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            cv2.putText(vis, f"Vol: {volume_cm3:.1f} cm³ (+/- {err_total:.1f}%)",
                        (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

            # === WRITE TO CSV ===
            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            with open(csv_filename, mode="a", newline="") as f:
                writer = csv.writer(f)
                writer.writerow([timestamp, frame_idx, marker_status_text,
                                 round(cm_per_pixel, 6), round(w_cm, 3), round(h_cm, 3),
                                 round(volume_cm3, 3), round(err_total, 3)])

    cv2.putText(vis, marker_status_text, (10, 70),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 200, 255), 2)

    cv2.imshow("Object Volume Measurement (Improved Error Model)", vis)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

print(f"\n✅ Data saved to: {csv_filename}")



✅ Data saved to: object_measurements_2025-11-07_18-57-30.csv
